# Proyecto: Extracción de datos de facturas eléctricas
## Objetivo:
Extraer y gestionar información clave de facturas eléctricas para optimizar la asesoría y gestión mensual.

In [ ]:
# Instalación de librerías necesarias
#!pip install langchain langchain-community langchain-experimental ollama PyMuPDF pydantic sqlalchemy psycopg[binary] faiss-cpu


### 📘 1. Importación de librerías necesarias

In [1]:
import os
import fitz  # PyMuPDF
import re
import json
from typing import List
from pydantic import BaseModel
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

/opt/anaconda3/envs/tfa_prompt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 📄 2. Cargar y leer texto desde el PDF

In [2]:
from pathlib import Path

import fitz  # PyMuPDF

ruta_facturas = Path("data/facturas/")

def extraer_texto_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf:
        texto_factura = ""
        for pagina in pdf:
            texto_factura += pagina.get_text()
    return texto_factura

for pdf_file in ruta_facturas.glob("*.pdf"):
    texto_factura = extraer_texto_pdf(pdf_file)
    print(f"\n📄 Archivo: {pdf_file.name}\n{'-'*40}")
    print(texto_factura[:500])
    print("-"*40)


📄 Archivo: Endesa Factura 08022024.pdf
----------------------------------------
VX10C02P-D-06/02/24 N0026249LNNNN
LARS TANKMAR TANKMAR
AVENIDA ONZE DE SETEMBRE 1 ESC-3 ATC-2
17255 BEGUR
GIRONA
Endesa Energía, S.A. Unipersonal. Inscrita en el Registro Mercantil de Madrid. Tomo 12.797, Libro 0, Folio 208,
 Sección 8ª, Hoja M-205.381, CIF A81948077. Domicilio Social: C/Ribera del Loira, nº60 28042 - Madrid.
Endesa Energía, S.A. Unipersonal.
CIF A81948077.
C/Ribera del Loira, nº 60 28042 - Madrid.
DATOS DE LA FACTURA
Nº factura: PMM401N0380480
Referencia: 012294855257/0412
Fech
----------------------------------------

📄 Archivo: Endesa Factura 06022025.pdf
----------------------------------------
VX10O020-D-11/02/25 N0016126LNNNN
LARS TANKMAR TANKMAR
AV ONZE DE SETEMBRE 1 3 ATC 2
17255 BEGUR
GERONA
Endesa Energía, S.A. Unipersonal. Inscrita en el Registro Mercantil de Madrid. Tomo 12.797, Libro 0, Folio 208,
 Sección 8ª, Hoja M-205.381, CIF A81948077. Domicilio Social: C/Ribera del Loira

### 🧾 3. Definir el esquema de los datos con Pydantic

In [3]:
from pydantic import BaseModel
from datetime import datetime

class FacturaElectrica(BaseModel):
    numero_factura: str
    fecha_emision: datetime
    periodo_inicio: datetime
    periodo_fin: datetime
    consumo_total_kwh: float
    potencia_punta_kw: float
    potencia_valle_kw: float
    importe_total: float


In [4]:
esquema_json = FacturaElectrica.schema_json(indent=2)
print(esquema_json)

/var/folders/8n/h66vxdts42ng4_hwl72sglsm0000gp/T/ipykernel_7533/4107199601.py:1: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  esquema_json = FacturaElectrica.schema_json(indent=2)


{
  "properties": {
    "numero_factura": {
      "title": "Numero Factura",
      "type": "string"
    },
    "fecha_emision": {
      "format": "date-time",
      "title": "Fecha Emision",
      "type": "string"
    },
    "periodo_inicio": {
      "format": "date-time",
      "title": "Periodo Inicio",
      "type": "string"
    },
    "periodo_fin": {
      "format": "date-time",
      "title": "Periodo Fin",
      "type": "string"
    },
    "consumo_total_kwh": {
      "title": "Consumo Total Kwh",
      "type": "number"
    },
    "potencia_punta_kw": {
      "title": "Potencia Punta Kw",
      "type": "number"
    },
    "potencia_valle_kw": {
      "title": "Potencia Valle Kw",
      "type": "number"
    },
    "importe_total": {
      "title": "Importe Total",
      "type": "number"
    }
  },
  "required": [
    "numero_factura",
    "fecha_emision",
    "periodo_inicio",
    "periodo_fin",
    "consumo_total_kwh",
    "potencia_punta_kw",
    "potencia_valle_kw",
    "impor

### 🧩 5. Fragmentación semántica y generación de embeddings

In [11]:
import re
from typing import List
from sentence_transformers import SentenceTransformer

# 📌 Cargar modelo de embeddings
modelo_embeddings = SentenceTransformer("all-MiniLM-L6-v2")

# Nueva función para fragmentar texto por secciones específicas
def chunk_por_secciones(texto: str, secciones: List[str]) -> List[str]:
    # Crear patrón regex que detecte los títulos de secciones
    patron = '|'.join([re.escape(sec) for sec in secciones])
    indices = [m.start() for m in re.finditer(patron, texto)]
    indices.append(len(texto))  # Para el último chunk
    
    chunks = []
    for i in range(len(indices)-1):
        chunk = texto[indices[i]:indices[i+1]].strip()
        if chunk:
            chunks.append(chunk)
    return chunks

# Lista de secciones típicas en factura eléctrica
secciones = [
    "Datos de la factura",
    "Número de factura",
    "Fecha de emisión",
    "Periodo de facturación",
    "Consumo eléctrico",
    "Consumo total",
    "Potencia contratada",
    "Total"
]

# Fragmentar el texto de la factura por secciones
fragmentos = chunk_por_secciones(texto_factura, secciones)

# 📌 Generar embeddings a partir de los fragmentos obtenidos
embeddings = modelo_embeddings.encode(fragmentos, convert_to_tensor=False)


### 🧩 5. Guardar embeddings en ChromaDB

In [7]:
!pip install chromadb

In [12]:
import chromadb

# Inicializar el cliente de ChromaDB (almacenamiento local por defecto)
client = chromadb.Client()

# Crear o obtener una colección
coleccion = client.get_or_create_collection("facturas_electricas")

# Guardar los embeddings y fragmentos en la colección
ids = [f"fragmento_{i}" for i in range(len(fragmentos))]
coleccion.add(
    embeddings=embeddings.tolist(),
    documents=fragmentos,
    ids=ids
)

print(f"Se han guardado {len(fragmentos)} fragmentos en ChromaDB.")

Se han guardado 3 fragmentos en ChromaDB.


In [1]:

from transformers import pipeline

# Conectar a ChromaDB
client = chromadb.PersistentClient(path="./chroma")
collection = client.get_or_create_collection(name="facturas_electricas")
# Cargar LLM (puedes cambiar por otro si usas API o HuggingFace Pipeline)
#qa_pipeline = pipeline("text-generation", model="gpt2", max_length=200)
qa_pipeline = pipeline("text-generation", model="sshleifer/tiny-gpt2", max_length=100)

def buscar_contexto(query, top_k=3):
    embedding = modelo_embeddings.encode(query).tolist()
    results = collection.query(
        query_embeddings=[embedding],
        n_results=top_k
    )
    return results['documents'][0]

def construir_prompt(contexto, pregunta):
    context_str = "\n".join(contexto)
    return f"""Usa la siguiente información de facturas eléctricas para responder la pregunta:
---
{context_str}
---
Pregunta: {pregunta}
Respuesta:"""

def generar_respuesta(prompt):
    output = qa_pipeline(prompt)
    return output[0]['generated_text'].replace(prompt, '').strip()

/opt/anaconda3/envs/tfa_prompt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'chromadb' is not defined

In [ ]:
!pip install streamlit

In [2]:
import streamlit as st
from rag_utils import buscar_contexto, construir_prompt, generar_respuesta

st.set_page_config(page_title="RAG - Facturas Eléctricas")

st.title("🧾 RAG para Análisis de Facturas Eléctricas")
st.markdown("Escribe tu consulta y usa el sistema RAG para obtener respuestas basadas en documentos.")

pregunta = st.text_input("Escribe tu pregunta aquí:", "")

if st.button("Consultar") and pregunta.strip():
    with st.spinner("Buscando en la base de conocimiento..."):
        contexto = buscar_contexto(pregunta)
        prompt = construir_prompt(contexto, pregunta)
        respuesta = generar_respuesta(prompt)

    st.subheader("🔍 Respuesta generada:")
    st.write(respuesta)

2025-05-27 00:07:21.952 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 00:07:21.953 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 00:07:22.054 
  command:

    streamlit run /opt/anaconda3/envs/tfa_prompt/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-27 00:07:22.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 00:07:22.055 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 00:07:22.055 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 00:07:22.056 Thread 'MainThread': missing ScriptRunContext! This warning can be 